In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
df_stations = pd.read_csv('/content/drive/Shareddrives/DATA 298/Datasets/Refined_datasets/ev_stations_data.csv')
df_stations.head()

,Unnamed: 0,station_name,street_address,city,zip,level1_count,level2_count,dc_fast_count,ev_network,geocode_status,...,restricted_access,year_opening,year_updated,year_confirmation,month_opening,month_updated,month_confirmation,day_opening,day_updated,day_confirmation
0,0,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,90015,0.0,12.0,0.0,Non-Networked,GPS,...,False,1995.0,2021,2020.0,8.0,3,11.0,30.0,11,9.0
1,1,California Air Resources Board,9530 Telstar Ave,El Monte,91731,0.0,3.0,0.0,Non-Networked,GPS,...,False,1996.0,2022,2022.0,10.0,9,9.0,15.0,14,14.0
2,2,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,92037,0.0,1.0,0.0,Non-Networked,GPS,...,False,1997.0,2022,2021.0,7.0,2,7.0,30.0,10,14.0
3,3,Galpin Motors,15421 Roscoe Blvd,Sepulveda,91343,0.0,2.0,0.0,Non-Networked,200-9,...,False,2012.0,2022,2021.0,12.0,2,12.0,11.0,10,9.0
4,4,Galleria at Tyler,1299 Galleria at Tyler,Riverside,92503,0.0,4.0,0.0,Non-Networked,GPS,...,False,1997.0,2021,2020.0,8.0,3,2.0,30.0,11,6.0


In [ ]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13618 entries, 0 to 13617
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                13618 non-null  int64  
 1   station_name              13618 non-null  object 
 2   street_address            13618 non-null  object 
 3   city                      13618 non-null  object 
 4   zip                       13618 non-null  object 
 5   level1_count              13618 non-null  float64
 6   level2_count              13618 non-null  float64
 7   dc_fast_count             13618 non-null  float64
 8   ev_network                13618 non-null  object 
 9   geocode_status            13618 non-null  object 
 10  latitude                  13618 non-null  float64
 11  longitude                 13618 non-null  float64
 12  last_confirmed_date       13594 non-null  object 
 13  date_updated              13618 non-null  object 
 14  date_o

In [ ]:
df_stations.drop(['Unnamed: 0','station_name','street_address','level1_count','dc_fast_count','ev_network',
                  'geocode_status','last_confirmed_date','date_updated','date_opened','ev_connector_types',
                  'access_type','facility_type','ev_pricing','renewable_source_on_site','restricted_access',
                  'year_updated','year_confirmation','month_opening','month_updated','month_confirmation',
                  'day_opening','day_updated','day_confirmation'],axis=1,inplace=True)

In [ ]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13618 entries, 0 to 13617
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          13618 non-null  object 
 1   zip           13618 non-null  object 
 2   level2_count  13618 non-null  float64
 3   latitude      13618 non-null  float64
 4   longitude     13618 non-null  float64
 5   year_opening  13617 non-null  float64
dtypes: float64(4), object(2)
memory usage: 638.5+ KB


In [ ]:
df_stations.loc[df_stations['year_opening'].isna()]

,city,zip,level2_count,latitude,longitude,year_opening
9071,Santa Maria,93454,10.0,34.951715,-120.413119,NaN


In [ ]:
df_stations = df_stations[df_stations.zip != 'G9N 0']
df_stations = df_stations[df_stations.zip != 'CA']

In [ ]:
df_stations = df_stations[df_stations['year_opening'].notna()]

In [ ]:
df_stations['zip'] = df_stations['zip'].astype(int)
df_stations['year_opening'] = df_stations['year_opening'].astype(int)
df_stations['level2_count'] = df_stations['level2_count'].astype(int)

In [ ]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13615 entries, 0 to 13617
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          13615 non-null  object 
 1   zip           13615 non-null  int64  
 2   level2_count  13615 non-null  int64  
 3   latitude      13615 non-null  float64
 4   longitude     13615 non-null  float64
 5   year_opening  13615 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 744.6+ KB


In [ ]:
df_stations.city = df_stations.city.astype('category').cat.codes

In [ ]:
df_stations.rename(columns={'year_opening': 'year'}, inplace=True)

In [ ]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13615 entries, 0 to 13617
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          13615 non-null  int16  
 1   zip           13615 non-null  int64  
 2   level2_count  13615 non-null  int64  
 3   latitude      13615 non-null  float64
 4   longitude     13615 non-null  float64
 5   year          13615 non-null  int64  
dtypes: float64(2), int16(1), int64(3)
memory usage: 664.8 KB


In [ ]:
df_vehicles = pd.read_csv('/content/drive/Shareddrives/DATA 298/Datasets/csv_datasets/Vehicle_Population_Zip_Code.csv')
df_vehicles

,Data Year,Fuel Type,ZIP,Number of Vehicles
0,2010,Diesel,90001,76
1,2010,Diesel,90002,52
2,2010,Diesel,90003,80
3,2010,Diesel,90004,100
4,2010,Diesel,90005,52
...,...,...,...,...
172574,2021,Propane,95648,2
172575,2021,Propane,95652,1
172576,2021,Propane,95814,1
172577,2021,Propane,95822,3


In [ ]:
df_vehicles.rename(columns={'Data Year': 'year', 'Fuel Type': 'fuel_type', 'ZIP': 'zip', 'Number of Vehicles':'num_vehicles'}, inplace=True)

In [ ]:
df_vehicles.head()

,year,fuel_type,zip,num_vehicles
0,2010,Diesel,90001,76
1,2010,Diesel,90002,52
2,2010,Diesel,90003,80
3,2010,Diesel,90004,100
4,2010,Diesel,90005,52


In [ ]:
df_vehicles = df_vehicles[df_vehicles.fuel_type == 'Electric']
df_vehicles = df_vehicles.reset_index()
df_vehicles

,index,year,fuel_type,zip,num_vehicles
0,2334,2010,Electric,90010,2
1,2335,2010,Electric,90012,5
2,2336,2010,Electric,90020,1
3,2337,2010,Electric,90021,1
4,2338,2010,Electric,90024,7
...,...,...,...,...,...
17222,161178,2021,Electric,96158,4
17223,161179,2021,Electric,96160,4
17224,161180,2021,Electric,96161,265
17225,161181,2021,Electric,96162,4


In [ ]:
df_vehicles.drop(['index','fuel_type'],axis=1,inplace=True)
df_vehicles

,year,zip,num_vehicles
0,2010,90010,2
1,2010,90012,5
2,2010,90020,1
3,2010,90021,1
4,2010,90024,7
...,...,...,...
17222,2021,96158,4
17223,2021,96160,4
17224,2021,96161,265
17225,2021,96162,4


In [ ]:
df = pd.merge(df_stations, df_vehicles, on=['zip','year'])
df

,city,zip,level2_count,latitude,longitude,year,num_vehicles
0,585,91343,2,34.221665,-118.468371,2012,2
1,339,90802,1,33.763700,-118.192000,2012,6
2,453,92262,2,33.824004,-116.543589,2011,2
3,477,94588,2,37.700187,-121.876172,2012,11
4,569,90405,3,34.010021,-118.495830,2019,591
...,...,...,...,...,...,...,...
11369,437,92057,0,33.241632,-117.291319,2018,180
11370,508,92376,0,34.075802,-117.370627,2018,30
11371,220,90247,0,33.890466,-118.299857,2021,252
11372,492,91739,0,34.112867,-117.533423,2018,271


In [ ]:
df['registered_evs_diff'] = df.groupby(['city', 'zip'])['num_vehicles'].diff()
df['charging_stations_diff'] = df.groupby(['city', 'zip'])['level2_count'].diff()
X = df[['latitude', 'longitude', 'year', 'num_vehicles', 'registered_evs_diff', 'charging_stations_diff']].values 

In [ ]:
y = df['level2_count'].values 

In [ ]:
col_mean = np.nanmean(X, axis=0)
print(col_mean)
inds = np.where(np.isnan(X))
X[inds] = np.take(col_mean, inds[1])

[ 3.57255211e+01 -1.19872561e+02  2.01990364e+03  6.00035783e+02
  2.04684480e+01 -7.07947150e-02]


In [ ]:
print(np.where(np.isnan(y)))

(array([], dtype=int64),)


In [ ]:
model = RandomForestRegressor() 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
model.fit(X_train,y_train)

RandomForestRegressor()

In [ ]:
prediction = model.predict(X_test)
mse = mean_squared_error(y_test, prediction)
rmse = mse**.5
print(mse)
print(rmse)

2.343329758241758
1.530793832703071


In [ ]:
model = RandomForestRegressor() 
tscv = TimeSeriesSplit(n_splits=5) 
for train_index, test_index in tscv.split(X): 
 X_train, X_test = X[train_index], X[test_index] 
 y_train, y_test = y[train_index], y[test_index] 
 model.fit(X_train, y_train) 
 print("Test set R^2: {:.2f}".format(model.score(X_test, y_test)))

Test set R^2: 0.68
Test set R^2: 0.45
Test set R^2: 0.43
Test set R^2: 0.59
Test set R^2: 0.39


In [ ]:
years = 10 
new_data = np.zeros((years, X.shape[1])) 
for i in range(years): 
 new_data[i, :] = [37.7749, -122.4194, 2023+i, i*1000, i*100,  i*10] 
predictions = model.predict(new_data) 
print(predictions)

[ 1.56 13.11 21.29 34.73 38.48 51.09 57.27 57.75 57.75 57.75]


In [ ]:
df_vehicles.to_csv('/content/drive/Shareddrives/DATA 298B/Datasets/model_ready_dataset.csv')